In [ ]:

import time, threading, pandas as pd, ipywidgets as iw
from ipyleaflet import Map, CircleMarker, Polyline, LayerGroup, WidgetControl, basemaps

# ------------------------------------------------------------------ #
# Load data
# ------------------------------------------------------------------ #
df = pd.read_csv("stream_demo_close.csv").sort_values("timestamp").reset_index(drop=True)

# ------------------------------------------------------------------ #
# helpers
# ------------------------------------------------------------------ #
def status_and_colour(true_id, claimed_id, pred_id):
    """Return ('normal'|'spoof'|'dark', colour hex)."""
    if pd.isna(claimed_id) or claimed_id == "":
        return "dark", "orange"
    if int(pred_id) == int(claimed_id):
        return "normal", "green"
    return "spoof", "red"

LABEL = {111111111: "V‑111", 222222222: "V‑222", 333333333: "V‑333"}

# ------------------------------------------------------------------ #
# ipyleaflet map & legend
# ------------------------------------------------------------------ #
m = Map(center=(df.lat.mean(), df.lon.mean()),
        zoom=9,
        basemap=basemaps.OpenStreetMap.Mapnik)
layer = LayerGroup(); m.add(layer)

legend_html = """
<div style="background:rgba(255,255,255,0.9);padding:6px 8px;
            border:1px solid #999;border-radius:4px;font-size:11px;">
<b>Legend</b><br>
<span style='color:green;'>●</span> Normal: fingerprint matches AIS claim<br>
<span style='color:red;'>●</span> Spoof&nbsp;: fingerprint ≠ AIS claim<br>
<span style='color:orange;'>●</span> Dark&nbsp;&nbsp;: no AIS claim detected
</div>
"""
m.add_control(WidgetControl(widget=iw.HTML(legend_html), position="bottomleft"))

# ------------------------------------------------------------------ #
# UI widgets
# ------------------------------------------------------------------ #
start_btn = iw.Button(description="▶ Start", button_style="success")
stop_btn  = iw.Button(description="■ Stop",  button_style="danger", disabled=True)
speed_sl  = iw.IntSlider(value=10, min=1, max=20, description="Speed ×")
ui_box    = iw.HBox([start_btn, stop_btn, speed_sl])

# ------------------------------------------------------------------ #
# playback thread (one trail per vessel)
# ------------------------------------------------------------------ #
markers = {}          # vessel_id -> CircleMarker (last point, for tooltip update)
trails  = {}          # vessel_id -> Polyline    (persistent path)
running = False

def playback():
    global running
    running = True
    layer.clear_layers(); markers.clear(); trails.clear()

    for _, row in df.iterrows():
        if not running:
            break

        true_id = int(row.true_vessel_id)
        pred_id = true_id                  # <‑‑ replace with model later
        status, colour = status_and_colour(true_id, row.claimed_id, pred_id)

        # create per‑vessel trail once
        if true_id not in trails:
            trails[true_id] = Polyline(locations=[], color="#6c6cff",
                                       weight=2, opacity=0.6)
            layer.add(trails[true_id])

        # extend trail
        trails[true_id].locations.append((row.lat, row.lon))

        # create a new marker for this ping
        popup_html = (
            f"<b>{LABEL[true_id]}</b><br>"
            f"{row.timestamp}<br>"
            f"True ID    : {true_id}<br>"
            f"Predicted ID: {pred_id}<br>"
            f"Claimed ID  : {row.claimed_id or '—'}<br>"
            f"Status     : {status}<hr style='margin:4px 0;'>"
            f"Wi‑Fi {row.wifi_ct}   BT {row.bt_ct}<br>"
            f"RSSI {row.mean_rssi} dBm"
        )

        marker = CircleMarker(location=(row.lat, row.lon),
                              radius=5, color=colour,
                              fill_color=colour, fill_opacity=0.9)
        marker.popup = iw.HTML(popup_html)
        layer.add(marker)

        # keep reference to last marker for tooltip update if desired
        markers[true_id] = marker

        time.sleep(1.0 / speed_sl.value)

    start_btn.disabled = False
    stop_btn.disabled  = True

def on_start(_):
    start_btn.disabled = True
    stop_btn.disabled  = False
    threading.Thread(target=playback, daemon=True).start()

def on_stop(_):
    global running
    running = False
    stop_btn.disabled = True  # playback thread re‑enables start

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)

# ------------------------------------------------------------------ #
# 5  display everything
# ------------------------------------------------------------------ #
display(m, ui_box)
# ▲▲ Notebook cell ▲▲

Map(center=[np.float64(34.34342008333333), np.float64(-76.51545908333333)], controls=(ZoomControl(options=['po…